In [1]:
# Загрузка библиотек необходимых для отрисовки графиков
import matplotlib
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.integrate import odeint, solve_ivp
from math import pi, log, radians, cos
import math

In [2]:
#функции нужные для дальнейшего решения
def calc_ws(gamma_wat: float) -> float:
    """
    Функция для расчета солесодержания в воде

    :param gamma_wat: относительная плотность по пресной воде с плотностью 1000 кг/м3, безразм.

    :return: солесодержание в воде, г/г
    """
    ws = (
            1 / (gamma_wat * 1000)
            * (1.36545 * gamma_wat * 1000 - (3838.77 * gamma_wat * 1000 - 2.009 * (gamma_wat * 1000) ** 2) ** 0.5)
        )
    # если значение отрицательное, значит скорее всего плотность ниже допустимой 992 кг/м3
    if ws > 0:
        return ws
    else:
        return 0


def calc_rho_w(ws: float, t: float) -> float:
    """
    Функция для расчета плотности воды в зависимости от температуры и солесодержания

    :param ws: солесодержание воды, г/г
    :param t: температура, К

    :return: плотность воды, кг/м3
    """
    rho_w = 1000 * (1.0009 - 0.7114 * ws + 0.2605 * ws ** 2) ** (-1)

    return rho_w / (1 + (t - 273) * 1e-4 * (0.269 * (t - 273) ** 0.637 - 0.8))


def calc_mu_w(ws: float, t: float, p: float) -> float:
    """
    Функция для расчета динамической вязкости воды по корреляции Matthews & Russel

    :param ws: солесодержание воды, г/г
    :param t: температура, К
    :param p: давление, Па

    :return: динамическая вязкость воды, сПз
    """
    a = (
            109.574
            - (0.840564 * 1000 * ws)
            + (3.13314 * 1000 * ws ** 2)
            + (8.72213 * 1000 * ws ** 3)
    )
    b = (
            1.12166
            - 2.63951 * ws
            + 6.79461 * ws ** 2
            + 54.7119 * ws ** 3
            - 155.586 * ws ** 4
    )

    mu_w = (
            a * (1.8 * t - 460) ** (-b)
            * (0.9994 + 0.0058 * (p * 1e-6) + 0.6534 * 1e-4 * (p * 1e-6) ** 2)
    )
    return mu_w


def calc_n_re(rho_w: float, q_ms: float, mu_w: float, d_tub: float) -> float:
    """
    Функция для расчета числа Рейнольдса

    :param rho_w: плотность воды, кг/м3
    :param q_ms: дебит жидкости, м3/с
    :param mu_w: динамическая вязкость воды, сПз
    :param d_tub: диаметр НКТ, м

    :return: число Рейнольдса, безразмерн.
    """
    v = q_ms / (np.pi * d_tub ** 2 / 4)
    return rho_w * v * d_tub / mu_w * 1000


def calc_ff_churchill(
        n_re: float,
        roughness: float,
        d_tub: float
) -> float:
    """
    Функция для расчета коэффициента трения по корреляции Churchill

    :param n_re: число Рейнольдса, безразмерн.
    :param roughness: шероховатость стен трубы, м
    :param d_tub: диаметр НКТ, м

    :return: коэффициент трения, безразмерн.
    """
    a = (-2.457 * np.log((7 / n_re) ** 0.9 + 0.27 * (roughness / d_tub))) ** 16
    b = (37530 / n_re) ** 16

    ff = 8 * ((8 / n_re) ** 12 + 1 / (a + b) ** 1.5) ** (1/12)
    return ff

def calc_ff_jain(
        n_re: float,
        roughness: float,
        d_tub: float
) -> float:
    """
    Функция для расчета коэффициента трения по корреляции Jain

    :param n_re: число Рейнольдса, безразмерн.
    :param roughness: шероховатость стен трубы, м
    :param d_tub: диаметр НКТ, м

    :return: коэффициент трения, безразмерн.
    """
    if n_re < 3000:
        ff = 64 / n_re
    else:
        ff = 1 / (1.14 - 2 * np.log10(roughness / d_tub + 21.25 / (n_re**0.9))) ** 2
    return ff

In [12]:
#Расчетная часть градиента давления,
def calc_grad_pres(p, h, gamma_water, q_liq, d_tub, angle, roughness, temp_grad, t_wh):
    
    psi = 1 / (10 ** 5)
    ws = calc_ws(gamma_water)
    t = t_wh + temp_grad * h
    rho_w = calc_rho_w(ws, t)
    mu_w = calc_mu_w(ws, t, p)
    n_re = calc_n_re(rho_w=rho_w, q_ms=q_liq, mu_w=mu_w, d_tub=d_tub)
    f = calc_ff_churchill(n_re=n_re, roughness=roughness, d_tub=d_tub)
    pres_gr = psi * (rho_w * 9.8 * np.cos(math.radians(angle)) - 0.815 * f * rho_w / d_tub ** 5 * q_liq ** 2)

    return pres_gr

In [16]:
#забойное давление
def main_func(data):

    gamma_water = data['gamma_water'] # относительная плотность по пресной воде с плотностью 1000 кг/м3, безразм
    md_vdp = data['md_vdp'] # измеренная глубина забоя скважины
    d_tub = data['d_tub'] # диаметр НКТ, м
    angle = data['angle']# угол наклона скважины к горизонтали, градусы
    roughness = data['roughness'] # шероховатость трубы, м
    p_wh = data['p_wh'] # давление на устье, Па
    t_wh = data['t_wh'] + 273 # температура на устье скважины, K
    temp_grad = data['temp_grad'] * 0.01 # геотермический градиент, К/м

    p_wf = []
    q_liq_1 = []
    for q in np.arange(1, 400, 10):
        q_liq = q / 86400
        sol = solve_ivp(
            calc_grad_pres,
            t_span=[0, md_vdp],
            y0=[p_wh],
            args=(gamma_water, q_liq, d_tub, angle, roughness, temp_grad, t_wh),
            t_eval=[md_vdp]
        )
        p_wf.append(sol.y[0][0])
        q_liq_1.append(int(q))

    print(q_liq_1)
    print(p_wf)
    result = {'p_wf': p_wf, 'q_liq': q_liq_1}
    return result
    
if __name__ == "__main__":

    with open('1.json') as file:
        data = json.load(file)
        sol = main_func(data)

    with open(r"output.json", "w", ) as file:
        json.dump(sol, file)

[1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391]
[268.89130516040444, 268.88906693825015, 268.88070435017323, 268.87024320242887, 268.8566864615083, 268.84012734843077, 268.82062560052975, 268.79822260981325, 268.77294863266894, 268.744826656716, 268.7138746691877, 268.6801070771215, 268.6435356418395, 268.60417011854145, 268.5620187083369, 268.5170883863645, 268.4693851454164, 268.41891418037454, 268.3656800302232, 268.3096866890306, 268.2509376938337, 268.1894361950564, 268.12518501353406, 268.05818668713835, 267.9884435092316, 267.9159575606391, 267.84073073642605, 267.7627647684759, 267.6820612446486, 267.598621625131, 267.51244725646734, 267.4235393836577, 267.3318991606444, 267.2375276594357, 267.14042587808177, 267.04059474767314, 266.9380351385032, 266.832747865515, 266.72473369313127, 266.6139933395503]
